In [17]:
import os
import numpy as np
from PIL import Image

def calculate_mean_std(folder_path):
    categories = ["pizza", "steak", "sushi"]
    sum_of_pixels = np.zeros(3)  # One for each color channel (RGB)
    sum_of_squares = np.zeros(3)
    total_pixels = 0
    
    for category in categories:
        category_path = os.path.join(folder_path, category)
        files = os.listdir(category_path)
        
        for file in files:
            image_path = os.path.join(category_path, file)
            image = Image.open(image_path)
            image_array = np.array(image).astype(np.float32)
            
            # Make sure the image is not grayscale
            if len(image_array.shape) == 3:
                sum_of_pixels += np.sum(image_array, axis=(0, 1))
                sum_of_squares += np.sum(np.square(image_array), axis=(0, 1))
                total_pixels += image_array.shape[0] * image_array.shape[1]
    
    # Calculating mean and std dev for each channel
    mean = sum_of_pixels / total_pixels
    variance = (sum_of_squares / total_pixels) - (mean ** 2)
    std_dev = np.sqrt(variance)
    
    return mean, std_dev

folder_path = "data/pizza_steak_sushi/20label/train"
mean, std_dev = calculate_mean_std(folder_path)
print("Mean RGB:", mean)
print("Standard Deviation RGB:", std_dev)
print("--------")
print("Mean RGB:", mean/255)
print("Standard Deviation RGB:", std_dev/255)


Mean RGB: [134.28807268 107.3587573   88.1992553 ]
Standard Deviation RGB: [72.20683078 70.88173764 73.10276441]
--------
Mean RGB: [0.52661989 0.42101473 0.34587943]
Standard Deviation RGB: [0.28316404 0.2779676  0.28667751]


In [14]:
[134.28807268/255, 107.3587573/255,   88.1992553/255 ]

[0.5266198928627451, 0.4210147345098039, 0.34587943254901965]

In [7]:
os.listdir("data/pizza_steak_sushi/20label")

['test', 'train', 'unlabeled']

In [68]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset

In [24]:

pss_mean = (0.52661989, 0.42101473, 0.34587943)
pss_std = (0.28316404, 0.2779676, 0.28667751)

In [30]:
transform_labeled = transforms.Compose(
    [
        transforms.Resize((64, 64)),
        transforms.RandomHorizontalFlip(),
        # transforms.RandomCrop(
        #     size=32, padding=int(32 * 0.125), padding_mode="reflect"
        # ),
        transforms.ToTensor(),
        transforms.Normalize(mean=pss_mean, std=pss_std),
    ]
)
transform_val = transforms.Compose(
    [
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=pss_mean, std=pss_std),
    ]
)


In [32]:

train_labeled_dataset = datasets.ImageFolder(
    root="C:\\Users\\can.michael\\Desktop\\others\\SSL\\FixMatch\\FixMatch-pytorch\\data\\pizza_steak_sushi\\20label\\train",
    transform=transform_labeled,
    target_transform=None,
)

In [33]:
len(train_labeled_dataset)

60

In [42]:
type(train_labeled_dataset[0][0]), train_labeled_dataset[0][1]

(torch.Tensor, 0)

In [49]:
from randaugment import RandAugmentMC

In [100]:
class PSS_Unlabeled(Dataset):
    def __init__(self, img_dir, transform):
        self.ids = os.listdir(img_dir)
        self.ids.sort()

        self.images_fps = [os.path.join(img_dir, image_id) for image_id in self.ids]

        self.transform = transform

    def __len__(self):
        return len(self.images_fps)

    def __getitem__(self, i):
        image = Image.open(self.images_fps[i])
        img_transformed = self.transform(image)

        return img_transformed
        # img_transform = transforms.Compose([transforms.PILToTensor()])
        # return img_transform(img_transformed[0]), img_transform(img_transformed[1])



In [101]:
class TransformFixMatch(object):
    def __init__(self, mean, std):
        self.weak = transforms.Compose(
            [
                transforms.Resize((64, 64)),
                transforms.RandomHorizontalFlip(),
            ]
        )
        self.strong = transforms.Compose(
            [
                transforms.Resize((64, 64)),
                transforms.RandomHorizontalFlip(),
                RandAugmentMC(n=2, m=10),
            ]
        )
        self.normalize = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)]
        )

    def __call__(self, x):
        weak = self.weak(x)
        strong = self.strong(x)
        return self.normalize(weak), self.normalize(strong)



In [102]:

train_unlabeled_dataset = PSS_Unlabeled(
    img_dir=r"C:\\Users\\can.michael\\Desktop\\others\SSL\\FixMatch\\FixMatch-pytorch\\data\\pizza_steak_sushi\\20label\\unlabeled",
    transform=TransformFixMatch(mean=pss_mean, std=pss_std),
)

In [105]:
len(train_unlabeled_dataset)

164